In [1]:
# !pip install --upgrade pip --quiet
# !pip install pyspark --quiet
# !pip install -U -q PyDrive --quiet
# !pip install numpy pandas --quiet

In [2]:
import sys

SCALA_VERSION = "2.12"
KAFKA_VERSION = "3.7.0"
PYSPARK_SCALA_VERSION = "2.12"
SPARK_VERSION = "3.5.1"

%env SCALA_VERSION=$SCALA_VERSION
%env KAFKA_VERSION=$KAFKA_VERSION
%env PYSPARK_SCALA_VERSION=$PYSPARK_SCALA_VERSION
%env SPARK_VERSION=$SPARK_VERSION

env: SCALA_VERSION=2.12
env: KAFKA_VERSION=3.7.0
env: PYSPARK_SCALA_VERSION=2.12
env: SPARK_VERSION=3.5.1


In [3]:
!bash start-kafka.sh
!tail -n 100 kafka/logs/server.log | grep -i "Kafka Server started"

Starting Kafka 3.7.0 with Scala 2.12
Starting Kafka 3.7.0
Stopping Kafka 3.7.0
Kafka 3.7.0 stopped
Cleaning up Kafka 3.7.0
Kafka Cluster ID: _LkT74aOROOJFCSeyg94zw
Starting Kafka 3.7.0
metaPropertiesEnsemble=MetaPropertiesEnsemble(metadataLogDir=Optional.empty, dirs={/tmp/kraft-combined-logs: EMPTY})
Formatting /tmp/kraft-combined-logs with metadata.version 3.7-IV4.
Starting Kafka 3.7.0 in /home/tudny/Documents/UW/MIMUW-master/PDD/PDD-PZ2/kafka
Kafka 3.7.0 started
Waiting for Kafka 3.7.0 to start 5/5
Kafka 3.7.0 started
[2024-06-12 12:12:51,728] INFO [KafkaRaftServer nodeId=1] Kafka Server started (kafka.server.KafkaRaftServer)


In [4]:
TOPIC1 = "topic1"
TOPIC2 = "topic2"
BOOTSTRAP_SERVER = "127.0.0.1:9092"

In [5]:
!./create-topics.sh {BOOTSTRAP_SERVER} {TOPIC1} {TOPIC2}

Creating topics...
Created topic topic1.
Created topic topic2.


In [6]:
%env TOPIC1=$TOPIC1
%env TOPIC2=$TOPIC2
%env BOOTSTRAP_SERVER=$BOOTSTRAP_SERVER
!rm -rf data_generator.log

env: TOPIC1=topic1
env: TOPIC2=topic2
env: BOOTSTRAP_SERVER=127.0.0.1:9092


In [7]:
%%script bash --bg --proc data_producer_script --out data_producer_output --err data_producer_error
echo "Starting data producer"
bash populate-topics.sh --topic1 $TOPIC1 --topic2 $TOPIC2 --bootstrap_server $BOOTSTRAP_SERVER
sleep 1

In [8]:
# await data_producer_error.read()

In [9]:
# await data_producer_output.read()

In [10]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

packages = [
    f"org.apache.spark:spark-sql-kafka-0-10_{PYSPARK_SCALA_VERSION}:{SPARK_VERSION}",
    f"org.apache.kafka:kafka-clients:{KAFKA_VERSION}",
]
# scc.stop(); spark.stop()
spark = (
    SparkSession.builder.master("local[*]")
    .config("spark.jars.packages", ",".join(packages))
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
    .appName("PDD PZ2")
    .getOrCreate()
)

scc = StreamingContext(spark.sparkContext, 1)

:: loading settings :: url = jar:file:/home/tudny/Documents/UW/MIMUW-master/PDD/PDD-PZ2/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tudny/.ivy2/cache
The jars for the packages stored in: /home/tudny/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a20021b7-e10d-415b-a9dc-e8604931cb5a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.7.0 in central
	found com.github.luben#zstd-jni;1.5.5-6 in central


In [11]:
!kafka/bin/kafka-console-consumer.sh --topic {TOPIC1} --max-messages 10 --bootstrap-server {BOOTSTRAP_SERVER}

{"time_point": 126156, "value": 0.09410680739826649}
{"time_point": 126157, "value": 0.5118207314609834}
{"time_point": 126158, "value": 0.11662237010053167}
{"time_point": 126159, "value": 0.47778886507832974}
{"time_point": 126160, "value": 0.09959175616732985}
{"time_point": 126161, "value": 0.8201016717563743}
{"time_point": 126162, "value": 0.7995540866076996}
{"time_point": 126163, "value": 0.5207838250880776}
{"time_point": 126164, "value": 0.06826958391362503}
{"time_point": 126166, "value": 0.09403094615406071}
Processed a total of 10 messages


In [12]:
!kafka/bin/kafka-console-consumer.sh --topic {TOPIC2} --max-messages 10 --bootstrap-server {BOOTSTRAP_SERVER}

{"time_point": 214274, "value": 0.30168400567203246}
{"time_point": 214275, "value": 3.084737043177089}
{"time_point": 214276, "value": 2.714406339279887}
{"time_point": 214277, "value": 1.8041776383006902}
{"time_point": 214278, "value": 2.4481351609087056}
{"time_point": 214279, "value": 1.1925350243393145}
{"time_point": 214280, "value": 2.24592130709534}
{"time_point": 214281, "value": 1.0229729390096782}
{"time_point": 214283, "value": 0.7497465561523857}
{"time_point": 214284, "value": 0.5707870853226493}
Processed a total of 10 messages


In [13]:
from pyspark.sql.types import StructType, FloatType, IntegerType, StructField
from pyspark.sql import functions as F
from pyspark.sql.streaming.state import GroupStateTimeout
import pandas as pd

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVER)
    .option("subscribe", f"{TOPIC1},{TOPIC2}")
    .option("startingOffsets", "earliest")
    .load()
)

# {"time_point": 53475, "value": 2.8696007107000634}
schema = StructType(
    [
        StructField("time_point", IntegerType(), True),
        StructField("value", FloatType(), True),
    ]
)

df = df.select(
    F.from_json(F.col("value").cast("string"), schema).alias("value"),
    F.col("topic"),
).select(
    F.col("topic"),
    F.col("value.time_point").alias("time_point"),
    F.col("value.value").alias("value"),
)

In [14]:
# def state_func(key, data, state):
#     item_count = (state.exists and state.get[0]) or 0
#     columns_str = ""
#     for el in data:
#         columns_str += ''.join([f"{k}, " for k in el])
#         item_count += len(el)
#     state.update((item_count,))
#     yield pd.DataFrame({
#         "id": [key[0]],
#         "countAsString": [str(item_count)],
#         "columns": [columns_str]
#     })
#
# samples = df.groupby("topic").applyInPandasWithState(
#     state_func,
#     outputStructType="id string, countAsString string, columns string",
#     stateStructType="len long",
#     outputMode="append",
#     timeoutConf=GroupStateTimeout.ProcessingTimeTimeout
# )
#
# query = samples.writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()
#
# query.awaitTermination(100)
# query.stop()

In [15]:
from typing import Iterable, Tuple, Final
from pyspark.sql.streaming.state import GroupState
from pyspark.sql.types import (
    ArrayType,
    FloatType,
    StringType,
    StructType,
    StructField,
    LongType,
)
import numpy as np
from scipy.stats import kstest

MAX_SAMPLE_SIZE: Final[int] = 1000
SAMPLE_SAVE_TIME: Final[int] = 100_000
COUNT_DUMP_MULTIPLIER: Final[int] = 10000

LAMBDA_FACTOR: Final[float] = 1e-5
P_IN: Final[float] = float(MAX_SAMPLE_SIZE) * LAMBDA_FACTOR


def run_kstests(sample: Iterable[float], previous_samples: Iterable[Iterable[float]]):
    return [
        kstest(sample, previous_sample).pvalue for previous_sample in previous_samples
    ]


def reservoir_sampling_state_func(
    key: Tuple[str],
    pdfs: Iterable[pd.DataFrame],
    state: GroupState,
):
    def pack_to_json(res_type, idx, topic, s):
        return f"""{{
            "type": "{res_type}",
            "idx": {idx},
            "topic": "{topic}",
            "s": {s:.2}
        }}"""

    (topic_name,) = key
    (
        current_sample,
        current_paper_sample,
        current_stream_size,
        next_save_moment,
        previous_samples,
        previous_paper_samples,
    ) = (state.exists and state.get) or ([], [], 0, 1, [], [])

    joined_pdfs = pd.concat(pdfs)
    joined_pdfs.sort_values("time_point", inplace=True)

    for _, time_point, value in joined_pdfs.values:
        if next_save_moment is None:
            raise ValueError(
                f"{state=} {next_save_moment=} {current_sample=} {current_stream_size=} {state.get=}"
            )
        if time_point >= next_save_moment * SAMPLE_SAVE_TIME:
            state.update(
                (
                    current_sample,
                    current_paper_sample,
                    current_stream_size,
                    next_save_moment + 1,
                    [current_sample] + previous_samples[:3],
                    [current_paper_sample] + previous_paper_samples[:3],
                )
            )
            yield pd.DataFrame(
                {
                    "topic": [topic_name],
                    "save_moment": [next_save_moment],
                    "sample": [current_sample],
                    "paper_sample": [current_paper_sample],
                    "kstest": [
                        [""]
                        + [
                            pack_to_json("RS", -idx - 1, topic_name, r)
                            for idx, r in enumerate(
                                run_kstests(current_sample, previous_samples)
                            )
                        ]
                        + [
                            pack_to_json("PS", -idx - 1, topic_name, r)
                            for idx, r in enumerate(
                                run_kstests(
                                    current_paper_sample, previous_paper_samples
                                )
                            )
                        ]
                    ],
                }
            )
            if not state.exists:
                raise ValueError(
                    f"State doesnt exist, but should {state=} {next_save_moment=} {current_sample=} {current_stream_size=} {state.get=}"
                )
            (
                current_sample,
                current_paper_sample,
                current_stream_size,
                next_save_moment,
                previous_samples,
                previous_paper_samples,
            ) = state.get

        current_stream_size += 1
        if len(current_sample) < MAX_SAMPLE_SIZE:
            current_sample.append(value)
        else:
            idx = np.random.randint(0, current_stream_size)
            if idx < MAX_SAMPLE_SIZE:
                current_sample[idx] = value

        if np.random.rand() < P_IN:
            f_x = len(current_paper_sample) / MAX_SAMPLE_SIZE
            if np.random.rand() < f_x:
                randomly_picked_idx = np.random.randint(0, len(current_paper_sample))
                current_paper_sample[randomly_picked_idx] = value
            else:
                current_paper_sample.append(value)

    state.update(
        (
            current_sample,
            current_paper_sample,
            current_stream_size,
            next_save_moment,
            previous_samples,
            previous_paper_samples,
        )
    )


reservoir_sampling_state_schema = StructType(
    [
        StructField("sample", ArrayType(FloatType()), True),
        StructField("paper_sample", ArrayType(FloatType()), True),
        StructField("stream_size", LongType(), True),
        StructField("next_save_moment", LongType(), True),
        StructField("previous_samples", ArrayType(ArrayType(FloatType())), True),
        StructField("previous_paper_samples", ArrayType(ArrayType(FloatType())), True),
    ]
)

reservoir_sampling_output_schema = StructType(
    [
        StructField("topic", StringType(), True),
        StructField("save_moment", LongType(), True),
        StructField("sample", ArrayType(FloatType()), True),
        StructField("paper_sample", ArrayType(FloatType()), True),
        StructField("kstest", ArrayType(StringType()), True),
    ]
)

samples = df.groupby("topic").applyInPandasWithState(
    reservoir_sampling_state_func,
    outputStructType=reservoir_sampling_output_schema,
    stateStructType=reservoir_sampling_state_schema,
    outputMode="append",
    timeoutConf=GroupStateTimeout.ProcessingTimeTimeout,
)

x = (
    samples.groupBy("save_moment")
    .agg(
        F.collect_list("topic").alias("topic"),
        F.collect_list("sample").alias("sample"),
        F.collect_list("paper_sample").alias("paper_sample"),
        F.collect_list("kstest").alias("kstest"),
    )
    .filter(F.size(F.col("topic")) == 2)
    .select(
        F.col("save_moment"),
        F.col("topic"),
        F.col("sample"),
        F.col("paper_sample"),
        F.col("kstest"),
    )
)

y = x.select(
    F.col("save_moment"),
    F.col("topic"),
    F.udf(lambda s: f"{run_kstests(s[0], [s[1]])[0]:.2}", StringType())(
        F.col("sample")
    ).alias("kstest-topics"),
    F.udf(lambda s: f"{run_kstests(s[0], [s[1]])[0]:.2}", StringType())(
        F.col("paper_sample")
    ).alias("kstest-topics-paper"),
    F.col("kstest"),
)

kstest_schema = StructType(
    [
        StructField("type", StringType(), True),
        StructField("idx", IntegerType(), True),
        StructField("topic", StringType(), True),
        StructField("s", FloatType(), True),
    ]
)


def empty_if_null(col):
    return F.when(F.isnull(col), F.lit("")).otherwise(col)


z = (
    y.select(
        F.col("save_moment"),
        F.col("topic"),
        F.col("kstest-topics"),
        F.col("kstest-topics-paper"),
        F.explode(F.col("kstest")).alias("kstest"),
    )
    .select(
        F.col("save_moment"),
        F.col("topic"),
        F.col("kstest-topics"),
        F.col("kstest-topics-paper"),
        F.explode(F.col("kstest")).alias("kstest"),
    )
    .select(
        F.col("save_moment"),
        F.col("topic"),
        F.when(F.length(F.col("kstest")) == F.lit(0), F.col("kstest-topics"))
        .otherwise(F.lit(""))
        .alias("kstest-topics"),
        F.when(F.length(F.col("kstest")) == F.lit(0), F.col("kstest-topics-paper"))
        .otherwise(F.lit(""))
        .alias("kstest-topics-paper"),
        F.from_json(F.col("kstest"), kstest_schema).alias("kstest"),
    )
    .select(
        F.col("save_moment").alias("Sample save moment"),
        F.col("topic").alias("Tested topics"),
        F.col("kstest-topics").alias("KS Test topics (RS)"),
        F.col("kstest-topics-paper").alias("KS Test topics (PS)"),
        empty_if_null(F.col("kstest.type")).alias("Sampling type"),
        empty_if_null(F.col("kstest.idx")).alias("KS test idx"),
        empty_if_null(F.col("kstest.topic")).alias("KS test topic"),
        empty_if_null(F.col("kstest.s")).alias("KS test p-value"),
    )
)

query = (
    z.writeStream.outputMode("update")
    .option("numRows", 1000)
    .option("truncate", False)
    .format("console")
    .start()
    
)

24/06/12 12:13:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-fa2ce04d-b5f0-4332-9220-e6aff5d46def. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/12 12:13:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [16]:
query.awaitTermination()
query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|Sample save moment|Tested topics   |KS Test topics (RS)|KS Test topics (PS)|Sampling type|KS test idx|KS test topic|KS test p-value|
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|1                 |[topic2, topic1]|0.0                |0.0                |             |           |             |               |
|1                 |[topic2, topic1]|0.0                |0.0                |             |           |             |               |
|2                 |[topic2, topic1]|0.0                |0.0                |             |           |             |               |
|2                 |[topic2, topic1]|                   |                   |RS           |-1      

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|Sample save moment|Tested topics   |KS Test topics (RS)|KS Test topics (PS)|Sampling type|KS test idx|KS test topic|KS test p-value|
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|5                 |[topic2, topic1]|8.6e-243           |1.9e-82            |             |           |             |               |
|5                 |[topic2, topic1]|                   |                   |RS           |-1         |topic2       |1.0            |
|5                 |[topic2, topic1]|                   |                   |RS           |-2         |topic2       |1.0            |
|5                 |[topic2, topic1]|                   |                   |RS           |-3      

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|Sample save moment|Tested topics   |KS Test topics (RS)|KS Test topics (PS)|Sampling type|KS test idx|KS test topic|KS test p-value|
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|7                 |[topic2, topic1]|2.4e-117           |0.0017             |             |           |             |               |
|7                 |[topic2, topic1]|                   |                   |RS           |-1         |topic2       |1.0            |
|7                 |[topic2, topic1]|                   |                   |RS           |-2         |topic2       |3.7E-12        |
|7                 |[topic2, topic1]|                   |                   |RS           |-3      

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|Sample save moment|Tested topics   |KS Test topics (RS)|KS Test topics (PS)|Sampling type|KS test idx|KS test topic|KS test p-value|
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|9                 |[topic2, topic1]|2e-79              |0.5                |             |           |             |               |
|9                 |[topic2, topic1]|                   |                   |RS           |-1         |topic2       |1.0            |
|9                 |[topic2, topic1]|                   |                   |RS           |-2         |topic2       |0.038          |
|9                 |[topic2, topic1]|                   |                   |RS           |-3      

/home/tudny/Documents/UW/MIMUW-master/PDD/PDD-PZ2/venv/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:394: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  return hypotest_fun_in(*args, **kwds)


...

-------------------------------------------
Batch: 8
-------------------------------------------
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|Sample save moment|Tested topics   |KS Test topics (RS)|KS Test topics (PS)|Sampling type|KS test idx|KS test topic|KS test p-value|
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|19                |[topic2, topic1]|2.8e-42            |0.54               |             |           |             |               |
|19                |[topic2, topic1]|                   |                   |RS           |-1         |topic2       |1.0            |
|19                |[topic2, topic1]|                   |                   |RS           |-2         |topic2       |1.0            |
|19                |[topic2, topic1]|                   |                   |RS           |-3      

/home/tudny/Documents/UW/MIMUW-master/PDD/PDD-PZ2/venv/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:394: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  return hypotest_fun_in(*args, **kwds)


-------------------------------------------
Batch: 9
-------------------------------------------
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|Sample save moment|Tested topics   |KS Test topics (RS)|KS Test topics (PS)|Sampling type|KS test idx|KS test topic|KS test p-value|
+------------------+----------------+-------------------+-------------------+-------------+-----------+-------------+---------------+
|21                |[topic2, topic1]|3.3e-37            |0.11               |             |           |             |               |
|21                |[topic2, topic1]|                   |                   |RS           |-1         |topic2       |1.0            |
|21                |[topic2, topic1]|                   |                   |RS           |-2         |topic2       |0.83           |
|21                |[topic2, topic1]|                   |                   |RS           |-3      

ERROR:root:KeyboardInterrupt while sending command.             (49 + 12) / 200]
Traceback (most recent call last):
  File "/home/tudny/Documents/UW/MIMUW-master/PDD/PDD-PZ2/venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tudny/Documents/UW/MIMUW-master/PDD/PDD-PZ2/venv/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 